In [17]:
import keras
from keras import LSTM, Dense, Input, concatenate, Reshape, Dropout, Bidirectional
from keras import Model, load_model
import numpy as np

Using TensorFlow backend.


ImportError: cannot import name 'LSTM' from 'keras' (/opt/anaconda3/lib/python3.7/site-packages/keras/__init__.py)

In [7]:
import os
start_token = " "

with open("names.txt") as f:
    names = f.read()[:-1].split('\n')
    names = [start_token+name for name in names]

In [8]:
def process_names(names,*,unwanted=['(', ')', '-', '.', '/']):
    names = [name.lower() for name in names]
    print("Total names:",len(names))
    chars = sorted(list(set(''.join(names))))

    def has_unwanted(word):
        for char in word:
            if char in unwanted:
                return True
        return False
    names = [name for name in names if not has_unwanted(name)]
    print("Amount of names after removing those with unwanted characters\n:",len(names))
    chars = [char for char in chars if char not in unwanted]
    print("Using the following characters:\n",chars)

    maxlen = max([len(name) for name in names])
    minlen = min([len(name) for name in names])
    print("Longest name is",maxlen,"characters long")
    print("Shortest name is",minlen,"characters long")
    
    # enchar indicates the end of the word
    # here it goes through unlikely-to-be-used characters to find one it can use
    endchars = '!£$%^&*()-_=+/?.>,<;:@[{}]#~'
    endchar = [ch for ch in endchars if ch not in chars][0]

    # ensures the character isn't already used & present in the training data
    assert(endchar not in chars)
    chars += endchar
    
    return names,chars


names,chars = process_names(names)

Total names: 83
Amount of names after removing those with unwanted characters
: 83
Using the following characters:
 [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']
Longest name is 12 characters long
Shortest name is 4 characters long


In [9]:
def make_sequences(names,seqlen):
    sequences, lengths, nextchars = [],[],[] # To have the model learn a more macro understanding, 
                                             # it also takes the word's length so far as input
    for name in names:
        if len(name) <= seqlen:
            sequences.append(name + chars[-1]*(seqlen - len(name)))
            nextchars.append(chars[-1])
            lengths.append(len(name))
        else:
            for i in range(0,len(name)-seqlen+1):
                sequences.append(name[i:i+seqlen])
                if i+seqlen < len(name):
                    nextchars.append(name[i+seqlen])
                else:
                    nextchars.append(chars[-1])
                lengths.append(i+seqlen)

    print(len(sequences),"sequences of length",seqlen,"made")
    
    return sequences,lengths,nextchars

seqlen = 4
sequences,lengths,nextchars = make_sequences(names,seqlen)

343 sequences of length 4 made


In [10]:
def make_onehots(*,sequences,lengths,nextchars,chars):
    x = np.zeros(shape=(len(sequences),len(sequences[0]),len(chars)), dtype='float32') # sequences
    x2 = np.zeros(shape=(len(lengths),max(lengths))) # lengths

    for i, seq in enumerate(sequences):
        for j, char in enumerate(seq):
            x[i,j,chars.index(char)] = 1.

    for i, l in enumerate(lengths):
        x2[i,l-1] = 1.

    y = np.zeros(shape=(len(nextchars),len(chars)))
    for i, char in enumerate(nextchars):
        y[i,chars.index(char)] = 1.
    
    return x,x2,y

x,x2,y = make_onehots(sequences=sequences,
                     lengths=lengths,
                     nextchars=nextchars,
                     chars=chars)

In [11]:
def get_dictchars(names,seqlen):
    dictchars = [{} for _ in range(seqlen)]

    for name in names:
        if len(name) < seqlen:
            continue
        dictchars[0][name[0]] = dictchars[0].get(name[0],0) + 1
        for i in range(1,seqlen):
            if dictchars[i].get(name[i-1],0) == 0:
                dictchars[i][name[i-1]] = {name[i]: 1}
            elif dictchars[i][name[i-1]].get(name[i],0) == 0:
                dictchars[i][name[i-1]][name[i]] = 1
            else:
                dictchars[i][name[i-1]][name[i]] += 1
    return dictchars
                
dictchars = get_dictchars(names,seqlen)
                

def generate_start_seq(dictchars):
    res = "" # The starting sequence will be stored here
    p = sum([n for n in dictchars[0].values()]) # total amount of letter occurences
    r = np.random.randint(0,p) # random number used to pick the next character
    tot = 0
    for key, item in dictchars[0].items():
        if r >= tot and r < tot + item:
            res += key
            break
        else:
            tot += item

    for i in range(1,len(dictchars)):
        ch = res[-1]
        if dictchars[i].get(ch,0) == 0:
            l = list(dictchars[i].keys())
            ch = l[np.random.randint(0,len(l))]
        p = sum([n for n in dictchars[i][ch].values()])
        r = np.random.randint(0,p)
        tot = 0
        for key, item in dictchars[i][ch].items():
            if r >= tot and r < tot + item:
                res += key
                break
            else:
                tot += item
    return res

In [12]:
def sample(preds,temperature=0.4):
    preds = np.asarray(preds).astype('float64')
    if temperature == 0:
        # Avoiding a division by 0 error
        return np.argmax(preds)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

def generate_name(model,start,*,chars=chars,temperature=0.4):
    maxlength = model.layers[3].input.shape[1]
    seqlen = int(model.layers[0].input.shape[1])
    result = start
    
    sequence_input = np.zeros(shape=(1,seqlen,len(chars)))
    for i, char in enumerate(start):
        sequence_input[0,i,chars.index(char)] = 1.
    
    length_input = np.zeros(shape=(1,maxlength))
    length_input[0,len(result)-1] = 1.
    
    prediction = model.predict(x=[sequence_input,length_input])[0]
    char_index = sample(prediction,temperature)
    while char_index < len(chars)-1 and len(result) < maxlength:
        result += chars[char_index]
        
        sequence_input = np.zeros(shape=(1,seqlen,len(chars)))
        for i, char in enumerate(result[(-seqlen):]):
            sequence_input[0,i,chars.index(char)] = 1.
        
        length_input[0,len(result)-2] = 0.
        length_input[0,len(result)-1] = 1.
        
        prediction = model.predict(x=[sequence_input,length_input])[0]
        char_index = sample(prediction,temperature)
    
    return result.title()

def generate_random_name(model,*,chars=chars,dictchars=dictchars,temperature=0.4):
    start = generate_start_seq(dictchars)
    return generate_name(model,start,chars=chars,temperature=temperature)

In [19]:
def make_model(x,x2,chars):
    inp1 = Input(shape=x.shape[1:]) # sequence input
    inp2 = Input(shape=x2.shape[1:]) # length input
    lstm = Bidirectional(LSTM(len(chars),activation='relu',dropout=0.3))(inp1)
    lstm2 = Bidirectional(LSTM(len(chars),dropout=0.3,go_backwards=True))(inp1)
    concat = concatenate([lstm,lstm2,inp2])
    dense = Dense(len(chars),activation='softmax')(concat)

    model = Model([inp1,inp2],dense)
    model.compile(optimizer='adam',loss='binary_crossentropy')
    return model

model = make_model(x,x2,chars)

In [20]:
def try_model(model,*,x=x,x2=x2,y=y,chars=chars,dictchars=dictchars,total_epochs=180,print_every=60,temperature=0.4,verbose=True):
    for i in range(total_epochs//print_every):
        history = model.fit([x,x2],y,
                            epochs=print_every,
                            batch_size=64,
                            validation_split=0.05,
                            verbose=0)
        if verbose:
            print("\nEpoch",(i+1)*print_every)
            print("First loss:            %1.4f" % (history.history['loss'][0]))
            print("Last loss:             %1.4f" % (history.history['loss'][-1]))
            print("First validation loss: %1.4f" % (history.history['val_loss'][0]))
            print("Last validation loss:  %1.4f" % (history.history['val_loss'][-1]))
            print("\nGenerating random names:")
            for _ in range(10):
                print(generate_random_name(model,chars=chars,dictchars=dictchars,temperature=temperature)) 
    if not verbose:
        print("Model training complete, here are some generated names:")
        for _ in range(20):
            print(generate_random_name(model,chars=chars,dictchars=dictchars,temperature=0.4))

In [ ]:
try_model(model)

Instructions for updating:
Use tf.cast instead.

Epoch 60
First loss:            0.1638
Last loss:             0.1157
First validation loss: 0.1645
Last validation loss:  0.1322

Generating random names:
 Graia
 Zaten
 Kaciell
 Jontea
 Kylee
 Kene
 Dasie
 Erai
 Raren
 Stehi

Epoch 120
First loss:            0.1142
Last loss:             0.0810
First validation loss: 0.1328
Last validation loss:  0.1300

Generating random names:
 Keal
 Cran
 Char
 Grin
 Liffany
 Eran
 Amin
 Jenin
 Kimin
 Darin
